In [230]:
import pandas as pd
import itertools as itt
import numbers as nb
import numpy as np
import re
from typing import Callable
from typing import Any

#warnings.filterwarnings('ignore')

idaho_data = pd.read_csv("Idaho/Idaho/idaho_data2.csv")

In [293]:
def pivot_dataset(
        df: pd.DataFrame,
        sample_id_column: str,
        per_sample_data: list[str],
        chemical_name_column: str,
        values_per_chemical: list[str],
        desired_chemical_names: list[str]
):

    df = df[df[chemical_name_column].isin(
        desired_chemical_names)]

    df = df.drop_duplicates(
        subset=[sample_id_column, chemical_name_column])

    df_pivoted = df.pivot(
        index=[sample_id_column, *per_sample_data], columns=[chemical_name_column], values=values_per_chemical)

    df_pivoted.columns = df_pivoted.columns.reorder_levels(order=[
        chemical_name_column, None])
    df_pivoted = df_pivoted.sort_index(axis=1, level=0)
    df_pivoted = df_pivoted.reset_index()  # ? May not be needed
    # Removes unnecessary data in the coulmns multiindex
    df_pivoted.columns.names = [None, None]
    return df_pivoted


dataset_pivoted = pivot_dataset(
    idaho_data,
    sample_id_column="SampleNumber",
    per_sample_data=["Latitude", "Longitude", "SampleDate"],
    chemical_name_column="CharName",
    values_per_chemical=["Amount", "UOM", "MinDetectLimit"],
    desired_chemical_names=["Chloride", "Sulfate", "Hardness", "Sodium", "Potassium", "Magnesium", "Calcium",
                            "Specific Conductance", "Total Dissolved Solids", "Water Temperature", "pH"],
)

dataset_pivoted

SampleNumber   Latitude   Longitude        SampleDate Calcium  \
                                                                      Amount   
0              0003ETAN655B  43.847830 -112.709326  29/03/2000 10:20    65.6   
1              0003ETAN655C  43.847830 -112.709326  29/03/2000 10:20     NaN   
2              0004ENRFIWDB  43.651806 -112.919948  20/04/2000 10:05     896   
3              0004ENRFIWDD  43.651806 -112.919948  20/04/2000 10:05     NaN   
4              0004ENRFSEWB  43.653774 -112.912298   20/04/2000 9:40     8.4   
...                     ...        ...         ...               ...     ...   
28678  IDEQ-2017-11-08-6159  43.376230 -116.550910   8/11/2017 14:32     NaN   
28679  IDEQ-2017-11-14-6160  43.717750 -116.998090  14/11/2017 10:55     NaN   
28680  IDEQ-2017-11-14-6161  43.713400 -116.902030  14/11/2017 11:40     NaN   
28681  IDEQ-2017-11-14-6162  43.697140 -117.020360  14/11/2017 12:17     NaN   
28682  IDEQ-2017-11-20-6163  43.897710 -111.157300  20/11/2017 10:50      66   

                           Chloride                       ... Sulfate  \
      MinDetectLimit   UOM   Amount MinDetectLimit   UOM  ...     UOM   
0                0.1  mg/l      NaN            NaN   NaN  ...     NaN   
1                NaN   NaN      411            0.9  mg/l  ...    mg/l   
2                0.1  mg/l      NaN            NaN   NaN  ...     NaN   
3                NaN   NaN     9713            0.9  mg/l  ...    mg/l   
4                0.1  mg/l      NaN            NaN   NaN  ...     NaN   
...              ...   ...      ...            ...   ...  ...     ...   
28678            NaN   NaN      NaN            NaN   NaN  ...     NaN   
28679            NaN   NaN      NaN            NaN   NaN  ...     NaN   
28680            NaN   NaN      NaN            NaN   NaN  ...     NaN   
28681            NaN   NaN      NaN            NaN   NaN  ...     NaN   
28682            NaN  mg/l     32.3            NaN  mg/l  ...    mg/l   

      Total Dissolved Solids                      Water Temperature  \
                      Amount MinDetectLimit   UOM            Amount   
0                        NaN            NaN   NaN               NaN   
1                       1320            1.0  mg/l               NaN   
2                        NaN            NaN   NaN               NaN   
3                      17100            1.0  mg/l               NaN   
4                        NaN            NaN   NaN               NaN   
...                      ...            ...   ...               ...   
28678                    NaN            NaN   NaN             15.79   
28679                    NaN            NaN   NaN             18.94   
28680                    NaN            NaN   NaN             15.42   
28681                    NaN            NaN   NaN             16.51   
28682                    NaN            NaN   NaN              9.02   

                              pH                      
      MinDetectLimit  UOM Amount MinDetectLimit  UOM  
0                NaN  NaN    NaN            NaN  NaN  
1                NaN  NaN    NaN            NaN  NaN  
2                NaN  NaN    NaN            NaN  NaN  
3                NaN  NaN    NaN            NaN  NaN  
4                NaN  NaN    NaN            NaN  NaN  
...              ...  ...    ...            ...  ...  
28678            NaN   °C   7.56            NaN   pH  
28679            NaN   °C   7.14            NaN   pH  
28680            NaN   °C   6.95            NaN   pH  
28681            NaN   °C   7.42            NaN   pH  
28682            NaN   °C    6.9            NaN   pH  

[28683 rows x 37 columns]

In [294]:
# Part 2 apply formatting to each cell and expand

desired_chemical_names=["Chloride", "Sulfate", "Hardness", "Sodium", "Potassium", "Magnesium", "Calcium",
                            "Specific Conductance", "Total Dissolved Solids", "Water Temperature", "pH"]

def transform_chemical_data(df: pd.DataFrame,
                            chem_name: str,
                            transform: Callable[[tuple[Any]], tuple[Any]],
                            input_row_names: list[str],
                            output_row_names: list[str]
                            ):

    input_row_multiindex = [(chem_name, i) for i in input_row_names]
    output_row_multiindex = [(chem_name, i) for i in output_row_names]

    df[output_row_multiindex] = df[input_row_multiindex].apply(
        lambda row: transform(*[row[i] for i in input_row_multiindex]),
        axis=1, result_type="expand"
    )

# This function should format a single amount value and output a float-prefix tuple.
def format_amount (erase_invalid : bool = False):

    def format_amount_func (amount, min_detection_limit):

        #if amount is already number
        if isinstance(amount,nb.Number):
            return ('=',float(amount))
        
        if isinstance(amount,str):
            #remove whitespace
            amount = ''.join(amount.split())
            amount_prefix = None

            #if amount matches prefix-numerical form
            if re.fullmatch(r'([<>=]?)[0-9]*(\.?)[0-9]+', amount):
                if re.match(r'[<>=]',amount):
                    amount_prefix = re.split(r'[<>=]', amount)
                    amount_prefix[1] = float(amount_prefix[1])
                    amount_prefix = (amount_prefix[0], amount_prefix[1])
                else:
                    amount_prefix = ("=",float(amount))
                return amount_prefix
            
            #if amount is below detection limit
            elif amount == "BDL" or amount == "ND":
                if min_detection_limit is not None:
                    return ("<", min_detection_limit)
                else:
                    return ("=", 0)

        #otherwise amount is invalid
        if erase_invalid:
            return (np.nan, np.nan)
        else:
            raise ValueError("Invalid Formatting - " + str(amount))
    
    return format_amount_func

# apply formatting to every chemical and sort columns
for chemical_name in desired_chemical_names:
    transform_chemical_data(dataset_pivoted, chemical_name, format_amount(True),
                            ["Amount", "MinDetectLimit"], ["Prefix", "Amount"])
    dataset_pivoted = dataset_pivoted.sort_index(axis=1)

dataset_pivoted


Calcium                             Chloride                        \
       Amount MinDetectLimit Prefix   UOM   Amount MinDetectLimit Prefix   
0        65.6            0.1      =  mg/l      NaN            NaN      =   
1         NaN            NaN      =   NaN    411.0            0.9      =   
2       896.0            0.1      =  mg/l      NaN            NaN      =   
3         NaN            NaN      =   NaN   9713.0            0.9      =   
4         8.4            0.1      =  mg/l      NaN            NaN      =   
...       ...            ...    ...   ...      ...            ...    ...   
28678     NaN            NaN      =   NaN      NaN            NaN      =   
28679     NaN            NaN      =   NaN      NaN            NaN      =   
28680     NaN            NaN      =   NaN      NaN            NaN      =   
28681     NaN            NaN      =   NaN      NaN            NaN      =   
28682    66.0            NaN      =  mg/l     32.3            NaN      =   

            Hardness                 ... Total Dissolved Solids        \
        UOM   Amount MinDetectLimit  ...                 Prefix   UOM   
0       NaN      NaN            NaN  ...                      =   NaN   
1      mg/l      NaN            NaN  ...                      =  mg/l   
2       NaN      NaN            NaN  ...                      =   NaN   
3      mg/l      NaN            NaN  ...                      =  mg/l   
4       NaN      NaN            NaN  ...                      =   NaN   
...     ...      ...            ...  ...                    ...   ...   
28678   NaN      NaN            NaN  ...                      =   NaN   
28679   NaN      NaN            NaN  ...                      =   NaN   
28680   NaN      NaN            NaN  ...                      =   NaN   
28681   NaN      NaN            NaN  ...                      =   NaN   
28682  mg/l      NaN            NaN  ...                      =   NaN   

      Water Temperature                                pH                 \
                 Amount MinDetectLimit Prefix  UOM Amount MinDetectLimit   
0                   NaN            NaN      =  NaN    NaN            NaN   
1                   NaN            NaN      =  NaN    NaN            NaN   
2                   NaN            NaN      =  NaN    NaN            NaN   
3                   NaN            NaN      =  NaN    NaN            NaN   
4                   NaN            NaN      =  NaN    NaN            NaN   
...                 ...            ...    ...  ...    ...            ...   
28678             15.79            NaN      =   °C   7.56            NaN   
28679             18.94            NaN      =   °C   7.14            NaN   
28680             15.42            NaN      =   °C   6.95            NaN   
28681             16.51            NaN      =   °C   7.42            NaN   
28682              9.02            NaN      =   °C   6.90            NaN   

                   
      Prefix  UOM  
0          =  NaN  
1          =  NaN  
2          =  NaN  
3          =  NaN  
4          =  NaN  
...      ...  ...  
28678      =   pH  
28679      =   pH  
28680      =   pH  
28681      =   pH  
28682      =   pH  

[28683 rows x 48 columns]

In [295]:
# remove units whitespace and convert to lowercase

units_index = list(itt.product(desired_chemical_names,["UOM"]))
dataset_pivoted[units_index] = dataset_pivoted[units_index].applymap(lambda x : "".join(x.strip().lower().split()) if isinstance(x,str) else x)

# report units

units = list(set(itt.chain.from_iterable(dataset_pivoted[chemical_name,"UOM"].unique() for chemical_name in desired_chemical_names)))
print(units)

# standardise units

def standardise_units (units_dict):
    def standardise_units_func (row):
        if not (row[unit_name] in units_dict.keys()):
            raise ValueError('Invalid units')
        row[amount_name] = units_dict[row[unit_name]](row[amount_name])
        return row
    return standardise_units_func

convert_to_standard = dict()
get_standard = dict()

# mass/volume concentration g/L
convert_to_standard['g/l'] = lambda x : x
convert_to_standard['mg/l'] = lambda x : x/1e3
convert_to_standard['ug/l'] = lambda x : x/1e6
convert_to_standard['µg/l'] = lambda x : x/1e6
convert_to_standard['ng/l'] = lambda x : x/1e9

# pH 
convert_to_standard['ph'] = lambda x : x

# temperature °c
convert_to_standard['°c'] = lambda x: x
convert_to_standard['°f'] = lambda x: 5*(x-32)/9

# conductivity us/cm
convert_to_standard['us/cm'] = lambda x: x
convert_to_standard['µs/cm'] = lambda x: x

# nan
convert_to_standard[np.nan] = lambda x: x


# apply conversion
for chemical in desired_chemical_names:
    dataset_pivoted[chemical] = dataset_pivoted[chemical].apply(standardise_units(convert_to_standard), axis = 1)


dataset_pivoted

['ph', 'ug/l', nan, 'mg/l', '°c', '°f', 'us/cm']


Calcium                             Chloride                        \
       Amount MinDetectLimit Prefix   UOM   Amount MinDetectLimit Prefix   
0      0.0656            0.1      =  mg/l      NaN            NaN      =   
1         NaN            NaN      =   NaN   0.4110            0.9      =   
2      0.8960            0.1      =  mg/l      NaN            NaN      =   
3         NaN            NaN      =   NaN   9.7130            0.9      =   
4      0.0084            0.1      =  mg/l      NaN            NaN      =   
...       ...            ...    ...   ...      ...            ...    ...   
28678     NaN            NaN      =   NaN      NaN            NaN      =   
28679     NaN            NaN      =   NaN      NaN            NaN      =   
28680     NaN            NaN      =   NaN      NaN            NaN      =   
28681     NaN            NaN      =   NaN      NaN            NaN      =   
28682  0.0660            NaN      =  mg/l   0.0323            NaN      =   

            Hardness                 ... Total Dissolved Solids        \
        UOM   Amount MinDetectLimit  ...                 Prefix   UOM   
0       NaN      NaN            NaN  ...                      =   NaN   
1      mg/l      NaN            NaN  ...                      =  mg/l   
2       NaN      NaN            NaN  ...                      =   NaN   
3      mg/l      NaN            NaN  ...                      =  mg/l   
4       NaN      NaN            NaN  ...                      =   NaN   
...     ...      ...            ...  ...                    ...   ...   
28678   NaN      NaN            NaN  ...                      =   NaN   
28679   NaN      NaN            NaN  ...                      =   NaN   
28680   NaN      NaN            NaN  ...                      =   NaN   
28681   NaN      NaN            NaN  ...                      =   NaN   
28682  mg/l      NaN            NaN  ...                      =   NaN   

      Water Temperature                                pH                 \
                 Amount MinDetectLimit Prefix  UOM Amount MinDetectLimit   
0                   NaN            NaN      =  NaN    NaN            NaN   
1                   NaN            NaN      =  NaN    NaN            NaN   
2                   NaN            NaN      =  NaN    NaN            NaN   
3                   NaN            NaN      =  NaN    NaN            NaN   
4                   NaN            NaN      =  NaN    NaN            NaN   
...                 ...            ...    ...  ...    ...            ...   
28678             15.79            NaN      =   °c   7.56            NaN   
28679             18.94            NaN      =   °c   7.14            NaN   
28680             15.42            NaN      =   °c   6.95            NaN   
28681             16.51            NaN      =   °c   7.42            NaN   
28682              9.02            NaN      =   °c   6.90            NaN   

                   
      Prefix  UOM  
0          =  NaN  
1          =  NaN  
2          =  NaN  
3          =  NaN  
4          =  NaN  
...      ...  ...  
28678      =   ph  
28679      =   ph  
28680      =   ph  
28681      =   ph  
28682      =   ph  

[28683 rows x 48 columns]